# Activate Virtual Environment

In [ ]:
# .\.venv\Scripts\Activate.ps1

# Methods to Import and Cache csv or xlsx

In [ ]:
import inspect
import re
import json
from pathlib import Path
import pandas as pd
def dprint(x): # https://stackoverflow.com/questions/32000934/print-a-variables-name-and-value/57225950#57225950
    frame = inspect.currentframe().f_back
    s = inspect.getframeinfo(frame).code_context[0]
    r = re.search(r"\((.*)\)", s).group(1)
    print("{} = {}".format(r,x))

def _pk1_path_for_file(file_path):
    """Return Path for .pk1 cache file stored next to the input file with same base name and extension '.pk1'."""
    p = Path(file_path) if not isinstance(file_path, Path) else file_path
    return p.with_suffix('.pk1')

def load_or_cache_excel(xlsx_path, read_kwargs=None, force_refresh=False):
    """Load DataFrame from a .pk1 cache next to the xlsx if present; otherwise read the xlsx and save the .pk1.
    Returns the DataFrame.
    read_kwargs: dict forwarded to pd.read_excel.
    force_refresh: if True, re-read the Excel and overwrite cache."""
    read_kwargs = read_kwargs or {}
    pk1 = _pk1_path_for_file(xlsx_path)
    if pk1.exists() and not force_refresh:
        try:
            df = pd.read_pickle(pk1)
            print(f'Loaded cache {pk1}')
            return df
        except Exception as e:
            print(f'Warning: failed to load {pk1} (will re-read Excel): {e}')
    # read Excel and attempt to save cache
    df = pd.read_excel(xlsx_path, **read_kwargs)
    try:
        df.to_pickle(pk1)
        print(f'Saved cache {pk1}')
    except Exception as e:
        print(f'Warning: could not save cache {pk1}: {e}')
    return df

def load_or_cache_csv(csv_path, read_kwargs=None, force_refresh=False):
    """Load DataFrame from a .pk1 cache next to the csv if present; otherwise read the csv and save the .pk1.
    Returns the DataFrame.
    read_kwargs: dict forwarded to pd.read_csv.
    force_refresh: if True, re-read the CSV and overwrite cache."""
    read_kwargs = read_kwargs or {}
    # Set default read parameters for our specific CSV format
    default_params = {
        'sep': ';',  # semicolon separated
        'decimal': ',',  # comma as decimal separator
        'parse_dates': ['TimeDate'],  # parse TimeDate column as datetime
    }
    # Update with any user-provided parameters
    read_kwargs = {**default_params, **read_kwargs}
    
    pk1 = _pk1_path_for_file(csv_path)
    if pk1.exists() and not force_refresh:
        try:
            df = pd.read_pickle(pk1)
            print(f'Loaded cache {pk1}')
            return df
        except Exception as e:
            print(f'Warning: failed to load {pk1} (will re-read CSV): {e}')
    # read CSV and attempt to save cache
    df = pd.read_csv(csv_path, **read_kwargs)
    try:
        df.to_pickle(pk1)
        print(f'Saved cache {pk1}')
    except Exception as e:
        print(f'Warning: could not save cache {pk1}: {e}')
    return df

In [ ]:
# Read CSV files (with pk1 cache next to each csv). Uses load_or_cache_csv from previous cell.
csv_file_path_AllaFlöden = r'c:\Users\chrini\OneDrive - Norconsult Group\Projekt\1097224_Pajala ARV\4 Underlag\04 Underkonsult, Sidokonsult\gefasystem.app.box.com\20251113\20251113_Alla_Floden.csv'
csv_file_path_Dosering = r'c:\Users\chrini\OneDrive - Norconsult Group\Projekt\1097224_Pajala ARV\4 Underlag\04 Underkonsult, Sidokonsult\gefasystem.app.box.com\20251113\20251113_Dosering.csv'
csv_file_path_Försedimentering = r'c:\Users\chrini\OneDrive - Norconsult Group\Projekt\1097224_Pajala ARV\4 Underlag\04 Underkonsult, Sidokonsult\gefasystem.app.box.com\20251113\20251113_Försedimentering.csv'
csv_file_path_MBBR = r'c:\Users\chrini\OneDrive - Norconsult Group\Projekt\1097224_Pajala ARV\4 Underlag\04 Underkonsult, Sidokonsult\gefasystem.app.box.com\20251113\20251113_MBBR.csv'
csv_file_path_Skivfilter = r'c:\Users\chrini\OneDrive - Norconsult Group\Projekt\1097224_Pajala ARV\4 Underlag\04 Underkonsult, Sidokonsult\gefasystem.app.box.com\20251113\20251113_Skivfilter.csv'
csv_file_path_Slambehandling = r'c:\Users\chrini\OneDrive - Norconsult Group\Projekt\1097224_Pajala ARV\4 Underlag\04 Underkonsult, Sidokonsult\gefasystem.app.box.com\20251113\20251113_Slambehandling.csv'

In [ ]:
# Load using the helper which places the .pk1 next to the csv with same base name
df_AllaFlöden= load_or_cache_csv(csv_file_path_AllaFlöden)
df_Dosering = load_or_cache_csv(csv_file_path_Dosering)
df_Försedimentering= load_or_cache_csv(csv_file_path_Försedimentering)
df_MBBR = load_or_cache_csv(csv_file_path_MBBR)
df_Skivfilter = load_or_cache_csv(csv_file_path_Skivfilter)
df_Slambehandling = load_or_cache_csv(csv_file_path_Slambehandling)

In [ ]:
# For multiple dataframes
for df in [df_AllaFlöden, df_Dosering, df_Försedimentering, df_MBBR, df_Skivfilter, df_Slambehandling]:
    df.rename(columns={'TimeDate' : 'DateTime'}, inplace=True)
    df.set_index('DateTime', inplace=True)
    df.drop(columns=['ID', 'TimeLength'], inplace=True, errors='ignore')
    # df.rename(columns=lambda x: x.replace('PajalaARV-AS01.AnalogIn.', ''), inplace=True)
    df.rename(columns=lambda x: x.split('.')[-1], inplace=True)
    # df.rename(columns=lambda x: x.replace('.*\.', '', regex=True), inplace=True)

# Renaming imported Dataframes

In [ ]:
# Rename dataframe columns
#TimeDate;TimeLength;FT10101;FT30101;FT72101;FT80101;LT23101
df_AllaFlöden.rename(columns={
    'FT10101': 'Inflöde FT10101 [m3/h]',
    'FT30101': 'MBBRflöde FT30101 [m3/h]',
    'FT72101': 'Utflöde FT72101 [m3/h]',
    'FT80101': 'Inflöde Extenslam FT80101 uncalib [m3/h]',
    'LT23101': 'Nivå Bräddning LT23101 [m]'}, inplace=True)
df_AllaFlöden.drop(columns=['Nivå Bräddning LT23101 [m]'], inplace=True, errors='ignore')

#TimeDate;TimeLength;FT90101;FT92101;FT92201;LT90101;LT91101;LT91201;LT91301
df_Dosering.rename(columns={
    'FT90101':'FällPAX FT90101 [l/h]',
    'FT92101':'FlockPoly FT92101 [l/h]',
    'FT92201':'SlamPuckPoly FT92201 [l/h]',
    'LT90101' : 'NivåPAX LT90101 [m]',
    'LT91101':'FlockPoly LT91101 [m]',
    'LT91201':'SlamPuckPoly LT91201 [m]',
    'LT91301':'SlamAvvatPoly LT91301 [m]',
    # 'FT92301':'AvvatPoly FT92301 [l/h]'
    }, inplace=True)

# TimeDate;TimeLength;FT10101;LT20103;LT21101;LT23101;P22101;P22201
df_Försedimentering.rename(columns={
    'FT10101':'Inflöde FT10101 [m3/h]',
    'LT20103':'FörsedUtlopp LT20103 [m]',
    'LT21101':'FlytSlam LT21101 [m]',
    'LT23101':'Nivå Bräddning LT23101 [m]',
    'P22101':'Returslam P22101 [1/0]',
    'P22201':'Returslam P22201 [1/0]'}, inplace=True)
df_Försedimentering.drop(columns=['Inflöde FT10101 [m3/h]'], inplace=True, errors='ignore')

# TimeDate;TimeLength;QT31101;LT31101;QT31102;QT31201;LT31201;QT31202
df_MBBR.rename(columns={
    'QT31101':'O2MBBR QT31101 [ppm]',
    'LT31101':'MBBR LT31101 [m]',
    'QT31102':'O2MBBR QT31102 [ppm]',
    'QT31201':'O2MBBR QT31201 [ppm]',
    'LT31201':'MBBR LT31201 [m]',
    'QT31202':'O2MBBR QT31202 [ppm]'}, inplace=True)

# TimeDate;TimeLength;LT70101;P70101_FQ1;PS70101;LT70102;LT70201;P70201_FQ1;PS70201;LT70202;LT71101
df_Skivfilter.rename(columns={
    'LT70101':'SFIn LT70101 [m]',
    'P70101_FQ1':'SFS P70101_FQ1 [Hz]',
    'PS70101':'SFSpol PS70101 [1/0]',
    'LT70102':'SFUt LT70102 [m]',
    'LT70201':'SFIn LT70201 [m]',
    'P70201_FQ1':'SFS P70201_FQ1 [Hz]',
    'PS70201':'SFS PS70201 [1/0]',
    'LT70202':'SFUt LT70202 [m]',
    'LT71101':'SFÖverSlam LT71101 [m]'}, inplace=True)

# TimeDate;TimeLength;FT80101;FT81101;CV80101;LT80102;M80103_FQ;P82104_FQ1;P81101_FQ;P81102_FQ;P81101;P81102;FT82101;P82101_FQ1;P82102_FQ1;P82101;P82102;LT81101;LT82101;LT84101;P84101_FQ1;P84102_FQ1;P84101;P84102
df_Slambehandling.rename(columns={
    'FT80101':'Inflöde Extenslam FT80101 uncalib [m3/h]',
    'FT81101':'PuckS FT81101 [m3/h]',
    'CV80101':'ExternS CV80101 [%]',
    'LT80102':'ExternS tråg LT80102 [m]',
    'M80103_FQ':'ExternRens M80103_FQ [Hz]',
    'P82104_FQ1':'AvvattnPolyMix P82104_FQ1 [Hz]',
    'P81101_FQ':'Puck P81101_FQ [Hz]',
    'P81102_FQ':'Puck P81102_FQ [Hz]',
    'P81101':'Puck P81101 [1/0]',
    'P81102':'Puck P81102 [1/0]',
    'FT82101':'SlamAvvattn FT82101 [m3/h]',
    'P82101_FQ1':'SlamAvvattn P82101_FQ1 [Hz]',
    'P82102_FQ1':'SlamAvvattn P82102_FQ1 [Hz]',
    'P82101': 'SlamAvvattn P82101 [1/0]',
    'P82102': 'SlamAvvattn P82102 [1/0]',
    'LT81101': 'Slamlager 1 LT81101 [m]',
    'LT82101':  'Slamlager 2 LT82101 [m]',
    'LT84101': 'Rejekt LT84101 [m]',
    'P84101_FQ1': 'Rejekt P84101_FQ1 [Hz]',
    'P84102_FQ1': 'Rejekt P84102_FQ1 [Hz]',
    'P84101': 'Rejekt P84101 [1/0]',
    'P84102': 'Rejekt P84102 [1/0]',
    }, inplace=True)

df_PlusNivåer = pd.DataFrame()
df_PlusNivåer['FörsedUtlopp LT20103 [+m]'] = df_Försedimentering['FörsedUtlopp LT20103 [m]'] + 164 #164.069
df_PlusNivåer['Nivå Bräddning LT23101 [+m]'] = df_Försedimentering['Nivå Bräddning LT23101 [m]'] + 163.100 #Golvnivå NWD
df_PlusNivåer['FlytSlam LT21101 [+m]'] = df_Försedimentering['FlytSlam LT21101 [m]'] + 160.560 #Golvnivå NWD
df_PlusNivåer['MBBR LT31101 [+m]'] = df_MBBR['MBBR LT31101 [m]'] + 158.510 #Golvnivå NWD
df_PlusNivåer['MBBR LT31201 [+m]'] = df_MBBR['MBBR LT31201 [m]'] + 158.510 #Golvnivå NWD
df_PlusNivåer['SFIn LT70101 [+m]'] = df_Skivfilter['SFIn LT70101 [m]'] +162.020 #Golvnivå NWD
df_PlusNivåer['SFUt LT70102 [+m]'] = df_Skivfilter['SFUt LT70102 [m]'] +162.020 #Golvnivå NWD
df_PlusNivåer['SFIn LT70201 [+m]'] = df_Skivfilter['SFIn LT70201 [m]'] +162.020 #Golvnivå NWD
df_PlusNivåer['SFUt LT70202 [+m]'] = df_Skivfilter['SFUt LT70202 [m]'] +162.020 #Golvnivå NWD
df_PlusNivåer['SFÖverSlam LT71101 [+m]'] = df_Skivfilter['SFÖverSlam LT71101 [m]'] +158.510 #Golvnivå NWD
df_PlusNivåer['ExternS tråg LT80102 [+m]'] = df_Slambehandling['ExternS tråg LT80102 [m]']/1000 +165.241 #Gissning centrum rejektrör
df_PlusNivåer['Slamlager 1 LT81101 [+m]'] = df_Slambehandling['Slamlager 1 LT81101 [m]'] + 160.050 #Golvnivå NWD
df_PlusNivåer['Slamlager 2 LT82101 [+m]'] = df_Slambehandling['Slamlager 2 LT82101 [m]'] + 160.050 #Golvnivå NWD
df_PlusNivåer['Rejekt LT84101 [+m]'] = df_Slambehandling['Rejekt LT84101 [m]'] + 163.954 #Golvnivå NWD

df_Slambehandling.drop(columns=['Inflöde Extenslam FT80101 uncalib [m3/h]'], inplace=True)
df_AllaFlöden['Inflöde Extenslam FT80101 [m3/h]'] = df_AllaFlöden['Inflöde Extenslam FT80101 uncalib [m3/h]'] + 0.486474692821503

df_Inflöde_FT10101 = df_AllaFlöden[['Inflöde FT10101 [m3/h]']]
df_Utflöde_FT72101 = df_AllaFlöden[['Utflöde FT72101 [m3/h]']]
df_Inflöde_Extenslam_FT80101 = df_AllaFlöden[['Inflöde Extenslam FT80101 [m3/h]']]
df_Nivå_Bräddning_LT23101 = df_Försedimentering[['Nivå Bräddning LT23101 [m]']]
df_MBBRflöde_FT30101 = df_AllaFlöden[['MBBRflöde FT30101 [m3/h]']]

# Merge the DataFrames on the DateTime index, aligning values
# Using merge instead of concat to handle any duplicate indices
# Merge all DataFrames sequentially
# Tidigare df_ax
# df_ax = pd.merge(df_Inflöde_FT10101, df_Utflöde_FT72101, left_index=True, right_index=True, how='outer')
# df_ax = pd.merge(df_ax, df_Inflöde_Extenslam_FT80101, left_index=True, right_index=True, how='outer')
# df_ax = pd.merge(df_ax, df_MBBRflöde_FT30101, left_index=True, right_index=True, how='outer')

# Nya df_ax
df_ax = df_AllaFlöden
df_ax = pd.merge(df_ax,df_Dosering, left_index=True, right_index=True, how='outer')
df_ax = pd.merge(df_ax,df_Försedimentering, left_index=True, right_index=True, how='outer')
df_ax = pd.merge(df_ax,df_MBBR, left_index=True, right_index=True, how='outer')
df_ax = pd.merge(df_ax,df_Skivfilter, left_index=True, right_index=True, how='outer')
df_ax = pd.merge(df_ax,df_Slambehandling, left_index=True, right_index=True, how='outer')

df_ax = df_ax.dropna(axis=1, how='all')

# Show the result
print("\nMerged DataFrame:")
dprint(df_ax.head())

# Check for any missing values after merge
print("\nMissing values in merged DataFrame:")
dprint(df_ax.isna().sum())

In [ ]:
# # Read CSV files (with pk1 cache next to each csv). Uses load_or_cache_csv from previous cell.
# csv_file_path_FT10101 = r'c:\Users\chrini\OneDrive - Norconsult Group\Projekt\1097224_Pajala ARV\4 Underlag\Mejl\20251104_1609_Re_ Malmberg Water i Yngsjö - ARV Pajala, åtgärd diverse styrpunkter_Kristofer Gramner\20251104\FT10101.csv'
# csv_file_path_FT30101 = r'c:\Users\chrini\OneDrive - Norconsult Group\Projekt\1097224_Pajala ARV\4 Underlag\Mejl\20251104_1609_Re_ Malmberg Water i Yngsjö - ARV Pajala, åtgärd diverse styrpunkter_Kristofer Gramner\20251104\FT30101.csv'
# csv_file_path_FT72101 = r'c:\Users\chrini\OneDrive - Norconsult Group\Projekt\1097224_Pajala ARV\4 Underlag\Mejl\20251104_1609_Re_ Malmberg Water i Yngsjö - ARV Pajala, åtgärd diverse styrpunkter_Kristofer Gramner\20251104\FT72101.csv'
# csv_file_path_FT80101 = r'c:\Users\chrini\OneDrive - Norconsult Group\Projekt\1097224_Pajala ARV\4 Underlag\Mejl\20251104_1609_Re_ Malmberg Water i Yngsjö - ARV Pajala, åtgärd diverse styrpunkter_Kristofer Gramner\20251104\FT80101.csv'
# csv_file_path_LT23101 = r'c:\Users\chrini\OneDrive - Norconsult Group\Projekt\1097224_Pajala ARV\4 Underlag\Mejl\20251104_1609_Re_ Malmberg Water i Yngsjö - ARV Pajala, åtgärd diverse styrpunkter_Kristofer Gramner\20251104\LT23101.csv'

# # Load using the helper which places the .pk1 next to the csv with same base name
# df_Inflöde_FT10101 = load_or_cache_csv(csv_file_path_FT10101) #Inflöde
# df_Utflöde_FT72101 = load_or_cache_csv(csv_file_path_FT72101) #Utflöde
# df_MBBRflöde_FT30101 = load_or_cache_csv(csv_file_path_FT30101) #MBBR-flöde
# df_Inflöde_Extenslam_FT80101 = load_or_cache_csv(csv_file_path_FT80101) #Inflöde Extenslam
# df_Nivå_Bräddning_LT23101 = load_or_cache_csv(csv_file_path_LT23101) #Utflöde Bräddning


In [ ]:
# # dprint(df_1.head())
# df_Inflöde_FT10101.rename(columns={'Val':'Inflöde FT10101'}, inplace=True)
# df_Inflöde_FT10101['DateTime'] = pd.to_datetime(df_Inflöde_FT10101['DateTime'])
# df_Inflöde_FT10101.drop(columns=['ID', 'TimeLength'], inplace=True)
# df_Inflöde_FT10101.set_index('DateTime', inplace=True)
# # dprint(df_1.head())


# # print(df_2.head())
# df_Utflöde_FT72101.rename(columns={'Val':'Utflöde FT72101'}, inplace=True)
# df_Utflöde_FT72101['DateTime'] = pd.to_datetime(df_Utflöde_FT72101['DateTime'])
# df_Utflöde_FT72101.drop(columns=['ID', 'TimeLength'], inplace=True)
# df_Utflöde_FT72101.set_index('DateTime', inplace=True)
# dprint(df_Utflöde_FT72101.head())

# df_Inflöde_Extenslam_FT80101.rename(columns={'TimeDate': 'DateTime', 'Val':'Inflöde Extenslam FT80101'}, inplace=True)
# df_Inflöde_Extenslam_FT80101['DateTime'] = pd.to_datetime(df_Inflöde_Extenslam_FT80101['DateTime'])
# df_Inflöde_Extenslam_FT80101.drop(columns=['ID', 'TimeLength'], inplace=True)
# df_Inflöde_Extenslam_FT80101.set_index('DateTime', inplace=True)
# dprint(df_Inflöde_Extenslam_FT80101.head())
# df_Inflöde_Extenslam_FT80101_before_zeroflow_calib = df_Inflöde_Extenslam_FT80101
# df_Inflöde_Extenslam_FT80101['Inflöde Extenslam FT80101'] = df_Inflöde_Extenslam_FT80101['Inflöde Extenslam FT80101'] + 0.486474692821503

# df_Nivå_Bräddning_LT23101.rename(columns={'TimeDate': 'DateTime', 'Val':'Nivå Bräddning LT23101'}, inplace=True)
# df_Nivå_Bräddning_LT23101['DateTime'] = pd.to_datetime(df_Nivå_Bräddning_LT23101['DateTime'])
# df_Nivå_Bräddning_LT23101.drop(columns=['ID', 'TimeLength'], inplace=True)
# df_Nivå_Bräddning_LT23101.set_index('DateTime', inplace=True)
# dprint(df_Nivå_Bräddning_LT23101.head())

# df_MBBRflöde_FT30101.rename(columns={'TimeDate': 'DateTime', 'Val':'MBBRflöde FT30101'}, inplace=True)
# df_MBBRflöde_FT30101['DateTime'] = pd.to_datetime(df_MBBRflöde_FT30101['DateTime'])
# df_MBBRflöde_FT30101.drop(columns=['ID', 'TimeLength'], inplace=True)
# df_MBBRflöde_FT30101.set_index('DateTime', inplace=True)
# dprint(df_MBBRflöde_FT30101.head())

# # Merge the DataFrames on the DateTime index, aligning values
# # Using merge instead of concat to handle any duplicate indices
# # Merge all DataFrames sequentially
# df_ax = pd.merge(df_Inflöde_FT10101, df_Utflöde_FT72101, left_index=True, right_index=True, how='outer')
# df_ax = pd.merge(df_ax, df_Inflöde_Extenslam_FT80101, left_index=True, right_index=True, how='outer')
# df_ax = pd.merge(df_ax, df_MBBRflöde_FT30101, left_index=True, right_index=True, how='outer')

# # Show the result
# print("\nMerged DataFrame:")
# dprint(df_ax.head())

# # Check for any missing values after merge
# print("\nMissing values in merged DataFrame:")
# dprint(df_ax.isna().sum())


In [ ]:
# Show all column names to verify what needs renaming
print("Columns in df_Inflöde_FT10101 [m3/h]:", df_Inflöde_FT10101.columns.tolist())
print("Columns in df_Utflöde_FT72101 [m3/h]:", df_Utflöde_FT72101.columns.tolist())
print("Columns in df_ax:", df_ax.columns.tolist())

In [ ]:
# Show DateTime indices where there are missing values
print("\nDateTime indices with missing FT10101:")
print(df_ax[df_ax['Inflöde FT10101 [m3/h]'].isna()].index.strftime('%Y-%m-%d %H:%M:%S').tolist())

print("\nDateTime indices with missing FT72101:")
print(df_ax[df_ax['Utflöde FT72101 [m3/h]'].isna()].index.strftime('%Y-%m-%d %H:%M:%S').tolist())

# Print summary of gaps
print("\nSummary of gaps:")
print(f"Total rows in merged DataFrame: {len(df_ax)}")
print(f"Rows with missing FT10101: {df_ax['Inflöde FT10101 [m3/h]'].isna().sum()}")
print(f"Rows with missing FT72101: {df_ax['Utflöde FT72101 [m3/h]'].isna().sum()}")
print(f"Rows with data in both columns: {len(df_ax) - df_ax.isna().any(axis=1).sum()}")

# Check and Remove Duplicate Timestamps
Before calculating moving averages, we need to identify and remove any duplicate timestamps from the source data.

In [ ]:
# Collect all duplicate timestamps before removing them
df_duplicate_timestamps = pd.DataFrame()

# Helper function to collect duplicates from a DataFrame
def collect_duplicates(df, df_name):
    """Collect duplicate rows and return them with a source column."""
    if df.index.duplicated().any():
        dup_mask = df.index.duplicated(keep=False)  # Mark ALL duplicates, not just subsequent ones
        dup_rows = df[dup_mask].copy()
        dup_rows['Source'] = df_name
        dup_rows['DuplicateGroup'] = dup_rows.index.astype(str)
        return dup_rows
    return pd.DataFrame()

# Check for and collect duplicate indices before removal
frames_to_check = {
    'df_ax': df_ax,
    'df_Inflöde_Extenslam_FT80101': df_Inflöde_Extenslam_FT80101,
    'df_Nivå_Bräddning_LT23101': df_Nivå_Bräddning_LT23101,
    'df_MBBRflöde_FT30101': df_MBBRflöde_FT30101
}

duplicate_collections = []
for name, df_frame in frames_to_check.items():
    if df_frame.index.duplicated().any():
        dup_count = df_frame.index.duplicated().sum()
        print(f"Warning: {name} has {dup_count} duplicate indices. Keeping first occurrence.")
        
        # Collect duplicates
        dup_df = collect_duplicates(df_frame, name)
        if not dup_df.empty:
            duplicate_collections.append(dup_df)
        
        # Remove duplicates from the original frame
        if name == 'df_ax':
            df_ax = df_ax[~df_ax.index.duplicated(keep='first')]
        elif name == 'df_Inflöde_Extenslam_FT80101':
            df_Inflöde_Extenslam_FT80101 = df_Inflöde_Extenslam_FT80101[~df_Inflöde_Extenslam_FT80101.index.duplicated(keep='first')]
        elif name == 'df_Nivå_Bräddning_LT23101':
            df_Nivå_Bräddning_LT23101 = df_Nivå_Bräddning_LT23101[~df_Nivå_Bräddning_LT23101.index.duplicated(keep='first')]
        elif name == 'df_MBBRflöde_FT30101':
            df_MBBRflöde_FT30101 = df_MBBRflöde_FT30101[~df_MBBRflöde_FT30101.index.duplicated(keep='first')]

# Combine all duplicate collections into one DataFrame
if duplicate_collections:
    df_duplicate_timestamps = pd.concat(duplicate_collections, axis=0)
    df_duplicate_timestamps = df_duplicate_timestamps.sort_values(['DuplicateGroup', 'Source'])
    print(f"\nTotal duplicate rows collected: {len(df_duplicate_timestamps)}")
    print(f"Unique duplicate timestamps: {df_duplicate_timestamps.index.nunique()}")
    
    # Save to CSV
    csv_path = 'duplicate_timestamps.csv'
    df_duplicate_timestamps.to_csv(csv_path)
    print(f"Duplicate timestamps saved to: {csv_path}")
else:
    print("\nNo duplicates found in any DataFrame.")

print(f"\nCleaned DataFrame sizes:")
print(f"  df_ax: {len(df_ax)} rows")
print(f"  df_Inflöde_Extenslam_FT80101: {len(df_Inflöde_Extenslam_FT80101)} rows")
print(f"  df_Nivå_Bräddning_LT23101: {len(df_Nivå_Bräddning_LT23101)} rows")
print(f"  df_MBBRflöde_FT30101: {len(df_MBBRflöde_FT30101)} rows")

In [ ]:
# Summary of duplicate timestamps
if not df_duplicate_timestamps.empty:
    print("=" * 60)
    print("DUPLICATE TIMESTAMPS SUMMARY")
    print("=" * 60)
    print(f"Total duplicate rows captured: {len(df_duplicate_timestamps)}")
    print(f"Number of unique duplicate timestamps: {df_duplicate_timestamps.index.nunique()}")
    
    print("\nDuplicates by source DataFrame:")
    source_counts = df_duplicate_timestamps.groupby('Source').size()
    for source, count in source_counts.items():
        print(f"  {source}: {count} rows")
    
    print("\nUnique timestamps that appear multiple times:")
    unique_dups = sorted(df_duplicate_timestamps.index.unique())
    for ts in unique_dups:
        print(f"  {ts.strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Show a sample of the data for one duplicate timestamp
    print("\nExample: Data for first duplicate timestamp")
    first_dup = unique_dups[0]
    sample = df_duplicate_timestamps[df_duplicate_timestamps.index == first_dup]
    print(sample.to_string())
    
    # Optionally save to CSV
    csv_path = 'duplicate_timestamps.csv'
    df_duplicate_timestamps.to_csv(csv_path)
    print(f"\nDuplicate timestamps saved to: {csv_path}")


In [ ]:
# Display duplicate timestamps if any were found
if 'df_duplicate_timestamps' in locals() and not df_duplicate_timestamps.empty:
    print(f"Duplicate timestamps DataFrame shape: {df_duplicate_timestamps.shape}")
    print("\nFirst few duplicate rows:")
    print(df_duplicate_timestamps.head(20))
    
    # Show summary by source
    print("\nDuplicates by source:")
    print(df_duplicate_timestamps.groupby('Source').size())
    
    # Show the actual duplicate timestamps
    print("\nUnique duplicate timestamps:")
    print(sorted(df_duplicate_timestamps.index.unique().strftime('%Y-%m-%d %H:%M:%S').tolist()))
else:
    print("No duplicates were found or df_duplicate_timestamps is empty.")
    print("\nNote: If you previously had duplicates, you may need to:")
    print("1. Re-run the data loading cells (cells 4-5)")
    print("2. Then re-run the moving averages cell to capture duplicates")


# Calculate Velocity for FT10101
Convert flow rate (m³/s) to velocity (m/s) using pipe inside diameter of 300 mm.

In [ ]:
import numpy as np

# Pipe inside diameter in meters
diameter_m = 0.300  # 300 mm = 0.3 m

# Calculate cross-sectional area: A = π × (d/2)²
area_m2 = np.pi * (diameter_m / 2) ** 2

print(f"Pipe inside diameter: {diameter_m * 1000} mm")
print(f"Cross-sectional area: {area_m2:.6f} m²")

# Calculate velocity: v = Q / A
# df_ax['Inflöde FT10101'] is in m³/s (assuming flow rate units)
# Velocity will be in m/s
df_Inflöde_FT10101_mps = pd.DataFrame(index=df_ax.index)
df_Inflöde_FT10101_mps['Inflöde FT10101 [m/s]'] = df_ax['Inflöde FT10101 [m3/h]'] / 3.6 / 1000 / area_m2

print(f"\nVelocity statistics:")
print(df_Inflöde_FT10101_mps['Inflöde FT10101 [m/s]'].describe())
print(f"\nFirst few values:")
print(df_Inflöde_FT10101_mps.head())

# Calculate Moving Averages for Velocity
Apply the same time-based moving average windows (1h, 24h, 7d) to the velocity data.

In [ ]:
# Calculate moving averages for velocity data
# Prepare container for velocity moving averages
df_velocity_ma = pd.DataFrame(index=df_Inflöde_FT10101_mps.index)

# Define the same time-based windows as used for flows
#windows_velocity = { '12min': '12min', '1h': '60min', '24h': '24h', '7d': '7D' }
windows_velocity = {'1h': '60min', '24h': '24h', '7d': '7D' }

for col in df_Inflöde_FT10101_mps.columns:
    for w_label, w_offset in windows_velocity.items():
        # Use time-based rolling which is robust to missing/irregular timestamps
        ma = df_Inflöde_FT10101_mps[col].rolling(w_offset, min_periods=1).mean()
        ma_col_name = f"{col}_MA_{w_label}"
        df_velocity_ma[ma_col_name] = ma

# Concat original velocity data with its moving averages
df_velocity_with_ma = pd.concat([df_Inflöde_FT10101_mps, df_velocity_ma], axis=1)

print("Velocity DataFrame with Moving Averages:")
print(f"Shape: {df_velocity_with_ma.shape}")
print(f"Columns: {df_velocity_with_ma.columns.tolist()}")
print("\nFirst few rows:")
print(df_velocity_with_ma.head())

# Compute Overflow Rate from Level Data

In [ ]:
# Compute overflow rate from level df_Nivå_Bräddning_LT23101 -> df_Utflöde_Bräddning_LT23101
import numpy as np

# Constants for the V-notch weir formula (Excel equivalent):
# =IF(E2<0.25;0;0.58*8/15*TAN(RADIANS(100)/2) * (E2-H)^(2.5) * SQRT(2*g) * 3600)
H_threshold = 0.25  # m, crest level H in the formula
angle_deg = 100.0   # degrees for the V-notch angle
g = 9.81            # m/s^2
df_Tröskelnivå_Bräddning_LT23101 = pd.DataFrame(
    data={'Tröskelnivå Bräddning LT23101 [m]': H_threshold},
    index=df_Nivå_Bräddning_LT23101.index
)

print(df_Nivå_Bräddning_LT23101['Nivå Bräddning LT23101 [m]'].describe())
# Prepare head above crest (clipped at 0)
head = (df_Nivå_Bräddning_LT23101['Nivå Bräddning LT23101 [m]'] - H_threshold).clip(lower=0.0)


# Precompute constant multiplier K = 0.58*8/15*TAN(RADIANS(100)/2)*SQRT(2*g)*3600
K = 0.58 * (8.0/15.0) * np.tan(np.radians(angle_deg)/2.0) * np.sqrt(2.0 * g) * 3600.0

# Flow [m3/h]
flow_m3h = K * np.power(head, 2.5)

# Build resulting DataFrame
df_Utflöde_Bräddning_LT23101 = pd.DataFrame(
    data={'Utflöde Bräddning LT23101 [m3/h]': flow_m3h},
    index=df_Nivå_Bräddning_LT23101.index
)

# Check for and remove duplicates in the overflow DataFrame
if df_Utflöde_Bräddning_LT23101.index.duplicated().any():
    dup_count = df_Utflöde_Bräddning_LT23101.index.duplicated().sum()
    print(f"Warning: df_Utflöde_Bräddning_LT23101 [m3/h] has {dup_count} duplicate indices. Keeping first occurrence.")
    df_Utflöde_Bräddning_LT23101 = df_Utflöde_Bräddning_LT23101[~df_Utflöde_Bräddning_LT23101.index.duplicated(keep='first')]

# Merge overflow flow into df_ax immediately so it's available for subsequent cells
df_ax = pd.merge(df_ax, df_Utflöde_Bräddning_LT23101, left_index=True, right_index=True, how='outer')

# Optional diagnostics
print('Computed df_Utflöde_Bräddning_LT23101 [m3/h]:')
dprint(df_Utflöde_Bräddning_LT23101.head())
print(f'df_Utflöde_Bräddning_LT23101 [m3/h]: {len(df_Utflöde_Bräddning_LT23101)} rows')
print(f'Merged into df_ax. df_ax now has {len(df_ax.columns)} columns.')


# Understanding pd.NA vs np.nan

## Key Differences

### `np.nan` (NumPy NaN)
- **Type**: `float` - it's literally the floating-point value "Not a Number"
- **Data Type**: Always has dtype `float64`
- **Compatibility**: Works with **matplotlib**, NumPy, and most numeric operations
- **Use Case**: Traditional missing value representation for numeric data
- **Example**: `np.nan + 5` returns `np.nan` (propagates through calculations)

### `pd.NA` (Pandas NA)
- **Type**: Special pandas object of type `NAType`
- **Data Type**: Can work with nullable integer, boolean, and string types (e.g., `Int64`, `boolean`)
- **Compatibility**: **NOT compatible with matplotlib** - causes `TypeError` when matplotlib tries to convert to float
- **Use Case**: Modern pandas nullable data types that distinguish "missing" from "NaN"
- **Example**: `pd.NA + 5` returns `pd.NA` (also propagates)

## Why We Use np.nan for Plotting

When passing data to matplotlib for visualization:
- Matplotlib internally converts arrays to float using `np.asarray(x, float)`
- `np.nan` is already a float → ✅ Works perfectly
- `pd.NA` is a special object → ❌ Raises `TypeError: float() argument must be a string or a real number, not 'NAType'`

## Best Practice
- Use `np.nan` for numeric DataFrames that will be plotted
- Use `pd.NA` for nullable integer/boolean columns where you need to distinguish between missing and zero
- **Always convert to np.nan before plotting**: `df = df.fillna(np.nan)`

## Example from This Notebook
```python
# Before fix: caused TypeError in matplotlib
df_ma_diff[diff_col] = pd.NA  # ❌ Breaks plotting

# After fix: works with matplotlib
df_ma_diff[diff_col] = np.nan  # ✅ Plots successfully

# Failsafe conversion before any plotting
df_flows = df_flows.fillna(np.nan)
df_flowdiff = df_flowdiff.fillna(np.nan)
```

# Overflow Level Interactive Chart

In [ ]:
from InteractivePlotWindow import InteractivePlotWindow as IPW
from PyQt6.QtWidgets import QApplication
df_Nivå_och_Tröskelnivå_Bräddning_LT23101 = pd.merge(df_Nivå_Bräddning_LT23101,df_Tröskelnivå_Bräddning_LT23101, 
                     left_index=True, right_index=True, how='outer')
if False:
    if __name__ == "__main__":
        import sys as _sys
        from IPython import get_ipython

        def _make_and_show():
            app = QApplication.instance() or QApplication(_sys.argv)
            
            # Calculate total number of series for settings file key
            n_series = len(df_Nivå_och_Tröskelnivå_Bräddning_LT23101.columns) + len(df_Utflöde_Bräddning_LT23101.columns)
            settings_key = f'Flöden och Flödesdiff ({n_series} series)'
            print(f"Chart settings key: {settings_key}")
            
            mainWin = IPW(df_axL = df_Nivå_och_Tröskelnivå_Bräddning_LT23101,
                                df_axL_Title = 'Nivå [m]', 
                                df_axR = df_Utflöde_Bräddning_LT23101, 
                                df_axR_Title = 'Bräddnivåflöde [m3/h]',
                                WindowTitle='Pajala ARV Bräddning - Nivå och Flöde',
                                settings_file=f'InteractivePlotWindow.json::{settings_key}'
                            )
            mainWin.show()
            # Keep references to avoid garbage collection in notebook kernels.
            
            # Store on the app and module globals so the objects persist after this function returns.
            try:
                app._pajala_mainWin = mainWin
            except Exception:
                pass
            globals()['_pajala_mainWin'] = mainWin
            globals()['_pajala_app'] = app
            return app

        # If running inside an IPython kernel (notebook), request IPython to enable the Qt event loop
        if 'ipykernel' in _sys.modules:
            try:
                ip = get_ipython()
                if ip is not None:
                    # enable GUI event loop integration; this avoids a blocking app.exec() call
                    ip.run_line_magic('gui', 'qt')
            except Exception:
                ip = None
            # Create and show window but do NOT call app.exec() - the event loop is managed by IPython
            app = _make_and_show()
            # Keep references in the IPython user namespace if available so users can interact with them
            if ip is not None:
                try:
                    ip.user_ns['_pajala_app'] = app
                    ip.user_ns['_pajala_mainWin'] = globals().get('_pajala_mainWin')
                except Exception:
                    # Fall back to module globals (already set by _make_and_show)
                    pass
        else:
            # Running as a script: start the blocking event loop
            app = _make_and_show()
            _sys.exit(app.exec())

# Interpolate Missing Values in df_ax
Create a new DataFrame with interpolated values to fill gaps in the time series data.

In [ ]:
# Selectively replace zeros with NaN (so zeros are treated as missing and interpolated)
# Run this BEFORE the interpolation cell. Then re-run the interpolation cell to apply.

import numpy as np
import pandas as pd

# 1) Choose which columns should treat 0 as missing
#    Edit this list to match the columns where zero means "no reading" (not a real zero)
columns_to_zero_to_nan = [
    'Inflöde FT10101 [m3/h]',
    'Utflöde FT72101 [m3/h]',
    'MBBRflöde FT30101 [m3/h]'
]
# 'Utflöde Bräddning LT23101' and 'Inflöde Extenslam FT80101' should contain zero-values legitimately, so we do NOT include it here

# Keep only those that exist and are numeric
target_cols = [c for c in columns_to_zero_to_nan if c in df_ax.columns and pd.api.types.is_numeric_dtype(df_ax[c])]
if not target_cols:
    print("No matching numeric columns found in df_ax for zero→NaN replacement.")
else:
    # 2) Optional: treat near-zero as zero (set to >0 for tolerance)
    zero_eps = 0.0  # e.g., 1e-6 to also treat tiny magnitudes as zero

    print("Replacing zeros with NaN in columns:", target_cols)
    total_replaced = 0
    for col in target_cols:
        s = df_ax[col]
        mask = (s == 0) if zero_eps == 0 else (s.abs() <= zero_eps)
        n = int(mask.sum())
        if n > 0:
            df_ax.loc[mask, col] = np.nan
        print(f"  {col}: replaced {n} value(s)")
        total_replaced += n

    print(f"Total replacements: {total_replaced}")
    print("Now re-run the interpolation cell to include these NaNs in interpolation.")


In [ ]:
# Create interpolated version of df_ax to fill missing values
# 1) Expand the time index to include missing timestamps
# 2) Interpolate interior gaps with SMOOTHING (windowed-average endpoints + linear)

import pandas as pd
import numpy as np

# Ensure a sorted, unique DateTime index for reliable reindexing
_df_sorted = df_ax.sort_index()
if _df_sorted.index.has_duplicates:
    _df_sorted = _df_sorted[~_df_sorted.index.duplicated(keep='first')]

# Infer frequency; fall back to 1 minute if unknown
_inferred_freq = pd.infer_freq(_df_sorted.index)
if _inferred_freq is None:
    _inferred_freq = '1min'
    print("Note: Could not infer index frequency. Assuming 1 minute.")

# Build complete time range and reindex to insert fully-missing timestamp rows
_full_index = pd.date_range(_df_sorted.index.min(), _df_sorted.index.max(), freq=_inferred_freq)
df_ax_full = _df_sorted.reindex(_full_index)

# Preserve the index name (e.g., 'DateTime') so downstream code and displays keep the heading
_idx_name = _df_sorted.index.name or 'DateTime'
df_ax_full.index.name = _idx_name

print(f"Inserted missing timestamps: {len(_full_index) - len(_df_sorted.index)}")
print(f"Full index length: {len(_full_index)} (from {_full_index.min()} to {_full_index.max()}, freq={_inferred_freq})")
print(f"Index name: {df_ax_full.index.name}")

# Copy to preserve a separate interpolated frame
df_ax_interpolated = df_ax_full.copy()
# Ensure the copy also carries the index name explicitly
df_ax_interpolated.index.name = _idx_name

# Report missing values before interpolation
print("\nMissing values BEFORE interpolation (after reindex):")
print(df_ax_interpolated.isna().sum())
print(f"\nTotal NaNs: {df_ax_interpolated.isna().sum().sum()}")

# ============================================================================
# SMOOTHED INTERPOLATION: Windowed-Average Endpoints + Linear
# ============================================================================
# Instead of connecting adjacent points, this method:
# 1. Finds each gap (sequence of NaN values)
# 2. Calculates mean of N minutes BEFORE the gap (start value)
# 3. Calculates mean of N minutes AFTER the gap (end value)
# 4. Linearly interpolates between these averaged endpoints
#
# This smooths out noise while preserving trends.

# Configuration
averaging_window = '15min'  # Window size for averaging before/after gaps
use_windowed_interpolation = True  # Set False to use standard methods below

print(f"\nInterpolation method: Windowed-Average Linear (window={averaging_window})")

if use_windowed_interpolation:
    _numeric_cols = [c for c in df_ax_interpolated.columns if pd.api.types.is_numeric_dtype(df_ax_interpolated[c])]
    
    for col in _numeric_cols:
        s = df_ax_interpolated[col].copy()
        
        # Identify valid data boundaries (for limit_area='inside')
        first_valid = s.first_valid_index()
        last_valid = s.last_valid_index()
        
        if first_valid is None or last_valid is None:
            continue  # Column is all NaN
        
        # Only interpolate within valid data range (limit_area='inside')
        interior_mask = (s.index >= first_valid) & (s.index <= last_valid)
        s_interior = s[interior_mask]
        
        # Find gaps (consecutive NaN sequences)
        is_nan = s_interior.isna()
        gap_starts = is_nan & ~is_nan.shift(1, fill_value=False)
        gap_ends = is_nan & ~is_nan.shift(-1, fill_value=False)
        
        gap_start_indices = s_interior.index[gap_starts]
        gap_end_indices = s_interior.index[gap_ends]
        
        # Process each gap
        for gap_start, gap_end in zip(gap_start_indices, gap_end_indices):
            # Get indices for averaging windows
            gap_start_loc = s_interior.index.get_loc(gap_start)
            gap_end_loc = s_interior.index.get_loc(gap_end)
            
            # Calculate time window for averaging
            try:
                window_offset = pd.Timedelta(averaging_window)
            except Exception:
                window_offset = pd.Timedelta(minutes=15)
            
            # BEFORE gap: average of data from (gap_start - window) to gap_start
            before_window_start = gap_start - window_offset
            before_data = s_interior[(s_interior.index >= before_window_start) & (s_interior.index < gap_start)]
            before_avg = before_data.mean() if len(before_data) > 0 and not before_data.isna().all() else None
            
            # AFTER gap: average of data from gap_end to (gap_end + window)
            after_window_end = gap_end + window_offset
            after_data = s_interior[(s_interior.index > gap_end) & (s_interior.index <= after_window_end)]
            after_avg = after_data.mean() if len(after_data) > 0 and not after_data.isna().all() else None
            
            # If we have both averaged endpoints, interpolate linearly between them
            if before_avg is not None and after_avg is not None:
                # Get all indices in the gap (including boundaries)
                gap_indices = s_interior.index[(s_interior.index >= gap_start) & (s_interior.index <= gap_end)]
                n_points = len(gap_indices)
                
                if n_points > 0:
                    # Create linear interpolation between averaged endpoints
                    interpolated_values = np.linspace(before_avg, after_avg, n_points + 2)[1:-1]
                    s_interior.loc[gap_indices] = interpolated_values
        
        # Update the main dataframe with interpolated values
        df_ax_interpolated.loc[interior_mask, col] = s_interior
        
    print(f"Applied windowed-average interpolation to {len(_numeric_cols)} columns")
    
else:
    # ALTERNATIVE STANDARD METHODS (if windowed interpolation is disabled)
    # Uncomment one of these:
    
    # Option 1: Cubic spline (smooth curves, good for most time series)
    interpolation_method = 'cubic'
    interpolation_order = None
    
    # Option 2: Polynomial (smoother, may overshoot)
    # interpolation_method = 'polynomial'
    # interpolation_order = 2  # 2 or 3 recommended
    
    # Option 3: Akima spline (smooth, less overshoot than cubic)
    # interpolation_method = 'akima'
    # interpolation_order = None
    
    # Option 4: Standard linear (point-to-point)
    # interpolation_method = 'linear'
    # interpolation_order = None
    
    print(f"\nUsing standard interpolation method: {interpolation_method}" + 
          (f" (order={interpolation_order})" if interpolation_order else ""))
    
    _numeric_cols = [c for c in df_ax_interpolated.columns if pd.api.types.is_numeric_dtype(df_ax_interpolated[c])]
    
    for col in _numeric_cols:
        try:
            if interpolation_method == 'polynomial' and interpolation_order:
                df_ax_interpolated[col] = df_ax_interpolated[col].interpolate(
                    method=interpolation_method,
                    order=interpolation_order,
                    limit_area='inside'
                )
            else:
                df_ax_interpolated[col] = df_ax_interpolated[col].interpolate(
                    method=interpolation_method,
                    limit_area='inside'
                )
        except Exception as e:
            print(f"Warning: Could not use {interpolation_method} for {col}, falling back to linear. Error: {e}")
            df_ax_interpolated[col] = df_ax_interpolated[col].interpolate(
                method='linear',
                limit_area='inside'
            )

# Report missing values after interpolation
print("\nMissing values AFTER interpolation:")
print(df_ax_interpolated.isna().sum())
print(f"\nTotal NaNs: {df_ax_interpolated.isna().sum().sum()}")

# Show some examples where interpolation filled gaps
print("\nExample: First few rows of original vs full vs interpolated data")
print("\nOriginal df_ax (head):")
print(df_ax.head(10))
print("\nReindexed df_ax_full (head):")
print(df_ax_full.head(10))
print("\nInterpolated df_ax_interpolated (head):")
print(df_ax_interpolated.head(10))

# --- Additional diagnostics requested previously: print rows with missing data (original df_ax) ---
print("\nRows with any missing values (original df_ax):")
rows_with_any_nan = df_ax[df_ax.isna().any(axis=1)]
print(f"Total rows with >=1 NaN: {len(rows_with_any_nan)} out of {len(df_ax)}")
print("Showing first 30 rows with NaN (use the DataFrame variable to inspect more):")
try:
    print(rows_with_any_nan.head(30).to_string())
except Exception:
    print(rows_with_any_nan.head(30))

# Identify interior (non-edge) NaNs per column on the original df_ax (informational)
interior_nan_rows = set()
interior_counts = {}
for col in df_ax.columns:
    if not pd.api.types.is_numeric_dtype(df_ax[col]):
        continue
    if not df_ax[col].isna().any():
        continue
    first_valid = df_ax[col].first_valid_index()
    last_valid = df_ax[col].last_valid_index()
    if first_valid is None or last_valid is None:
        continue
    edge_mask = (df_ax.index < first_valid) | (df_ax.index > last_valid)
    interior_mask = (~edge_mask) & df_ax[col].isna()
    count_interior = int(interior_mask.sum())
    if count_interior > 0:
        interior_nan_rows.update(df_ax.index[interior_mask].tolist())
    interior_counts[col] = count_interior

print("\nInterior NaN rows (within valid span for each column):", len(interior_nan_rows))
if interior_nan_rows:
    sample_idx = sorted(list(interior_nan_rows))[:50]
    print("Showing up to 50 interior-NaN rows:")
    try:
        print(df_ax.loc[sample_idx].to_string())
    except Exception:
        print(df_ax.loc[sample_idx])
else:
    print("No interior NaN rows detected; NaNs appear at edges for their respective columns.")

print("\nInterior NaN counts per numeric column:")
for col, cnt in interior_counts.items():
    print(f"  {col}: {cnt}")

In [ ]:
# Edge NaN summary after interpolation
import pandas as pd

# Require df_ax_interpolated created in interpolation cell
if 'df_ax_interpolated' not in globals():
    print("df_ax_interpolated is not defined. Run the interpolation cell first.")
else:
    leading_counts = {}
    trailing_counts = {}
    interior_remaining_counts = {}
    for col in df_ax_interpolated.columns:
        if not pd.api.types.is_numeric_dtype(df_ax_interpolated[col]):
            continue
        s = df_ax_interpolated[col]
        if not s.isna().any():
            leading_counts[col] = 0
            trailing_counts[col] = 0
            interior_remaining_counts[col] = 0
            continue
        first_valid = s.first_valid_index()
        last_valid = s.last_valid_index()
        # If no valid data at all
        if first_valid is None or last_valid is None:
            leading_counts[col] = int(s.isna().sum())
            trailing_counts[col] = 0
            interior_remaining_counts[col] = 0
            continue
        leading_mask = (df_ax_interpolated.index < first_valid)
        trailing_mask = (df_ax_interpolated.index > last_valid)
        interior_mask = (~leading_mask) & (~trailing_mask)
        leading_counts[col] = int(s[leading_mask].isna().sum())
        trailing_counts[col] = int(s[trailing_mask].isna().sum())
        # Any NaNs that remain interior (should be zero with limit_area='inside')
        interior_remaining_counts[col] = int(s[interior_mask].isna().sum())

    print("Edge NaN Summary (post-interpolation):")
    print("Column | Leading NaNs | Trailing NaNs | Interior NaNs (remaining)")
    for col in leading_counts.keys():
        print(f"{col} | {leading_counts[col]} | {trailing_counts[col]} | {interior_remaining_counts[col]}")

    total_leading = sum(leading_counts.values())
    total_trailing = sum(trailing_counts.values())
    total_interior_remain = sum(interior_remaining_counts.values())
    print("\nTotals:")
    print(f"Leading edge NaNs: {total_leading}")
    print(f"Trailing edge NaNs: {total_trailing}")
    print(f"Interior remaining NaNs: {total_interior_remain}")

    # Percentages relative to all NaNs left
    total_remaining = total_leading + total_trailing + total_interior_remain
    if total_remaining > 0:
        pct_leading = 100 * total_leading / total_remaining
        pct_trailing = 100 * total_trailing / total_remaining
        pct_interior = 100 * total_interior_remain / total_remaining
        print(f"\nPercent distribution of remaining NaNs:")
        print(f"Leading: {pct_leading:.2f}% | Trailing: {pct_trailing:.2f}% | Interior: {pct_interior:.2f}%")
    else:
        print("\nNo NaNs remain after interpolation.")


# Calculate Moving Averages and Flow Plant Balance
Now that duplicates have been removed, we can safely calculate moving averages with different window sizes to smooth the time series data.

In [ ]:
df_ax = df_ax_interpolated.copy()
print(f"df_ax after interpolation has {len(df_ax)} rows and {len(df_ax.columns)} columns.")

In [ ]:
# Calculate total inflow and outflow
df_ax['Total Inflöde [m3/h]'] = df_ax['Inflöde FT10101 [m3/h]'] + df_ax['Inflöde Extenslam FT80101 [m3/h]']
df_ax['Total Utflöde [m3/h]'] = df_ax['Utflöde FT72101 [m3/h]'] + df_ax['Utflöde Bräddning LT23101 [m3/h]']
dprint(df_ax[['Total Inflöde [m3/h]', 'Total Utflöde [m3/h]']].head())

In [ ]:
# Calculate moving averages for each column (time-based windows)
# Each row in df_ax represents 1 minute, so use time-based rolling windows
import pandas as pd
import numpy as np

# Note: df_Utflöde_Bräddning_LT23101 was already merged into df_ax in the overflow calculation cell

# Prepare container for moving averages
df_ma = pd.DataFrame(index=df_ax.index)

# Define time-based windows (labels -> pandas offset strings)
#windows = { '12min': '12min', '1h': '60min', '24h': '24h', '7d': '7D' }
windows = { '1h': '60min', '24h': '24h', '7d': '7D' }

# Only compute MAs for numeric columns
numeric_cols = [c for c in df_ax.columns if pd.api.types.is_numeric_dtype(df_ax[c])]
for col in numeric_cols:
    for w_label, w_offset in windows.items():
        ma = df_ax[col].rolling(w_offset, min_periods=1).mean()
        df_ma[f"{col}_MA_{w_label}"] = ma

# Compute differences (Inflöde - Utflöde - Bräddning) for raw data and each moving-average window
# Source column names in df_ax are 'Inflöde FT10101', 'Utflöde FT72101', and 'Utflöde Bräddning LT23101'
df_ma_diff = pd.DataFrame(index=df_ax.index)
inflow_main = 'Inflöde FT10101 [m3/h]'
outflow_main = 'Utflöde FT72101 [m3/h]'
inflow_externslam = 'Inflöde Extenslam FT80101 [m3/h]'
outflow_bräddning = 'Utflöde Bräddning LT23101 [m3/h]'

required_raw = [inflow_main, outflow_main, inflow_externslam, outflow_bräddning]
missing_raw = [c for c in required_raw if c not in df_ax.columns]

# Raw instantaneous ratio (Diff_1min)
if not missing_raw:
    total_inflow_raw = df_ax[inflow_main] + df_ax[inflow_externslam]
    total_outflow_raw = df_ax[outflow_main] + df_ax[outflow_bräddning]
    denom_raw = total_inflow_raw.replace(0, np.nan)
    df_diff_raw = (total_inflow_raw - total_outflow_raw) / denom_raw
    df_diff_raw.name = 'Diff_1min'
else:
    df_diff_raw = pd.Series(data=np.nan, index=df_ax.index, name='Diff_1min')
    print(f"Warning: cannot compute Diff_1min, missing columns: {missing_raw}")

# Moving-average based ratios
missing_ma_warnings = []
for w_label in windows.keys():
    col_inflow_main = f"{inflow_main}_MA_{w_label}"
    col_outflow_main = f"{outflow_main}_MA_{w_label}"
    col_inflow_externslam = f"{inflow_externslam}_MA_{w_label}"
    col_outflow_bräddning = f"{outflow_bräddning}_MA_{w_label}"
    needed = [col_inflow_main, col_outflow_main, col_inflow_externslam, col_outflow_bräddning]
    if all(c in df_ma.columns for c in needed):
        total_inflow_ma = df_ma[col_inflow_main] + df_ma[col_inflow_externslam]
        total_outflow_ma = df_ma[col_outflow_main] + df_ma[col_outflow_bräddning]
        denom_ma = total_inflow_ma.replace(0, np.nan)
        df_ma_diff[f"Diff_MA_{w_label}"] = (total_inflow_ma - total_outflow_ma) / denom_ma
    else:
        df_ma_diff[f"Diff_MA_{w_label}"] = np.nan
        missing_set = [c for c in needed if c not in df_ma.columns]
        missing_ma_warnings.append((w_label, missing_set))

if missing_ma_warnings:
    for w_label, miss in missing_ma_warnings:
        print(f"Warning: cannot compute Diff_MA_{w_label} because missing MA columns: {miss}")

# Concat original df_ax and its moving averages (left axis)
df_ax_ma = pd.concat([df_ax, df_ma], axis=1)

# Right axis: raw diff plus MA diffs
df_flowdiff = pd.concat([df_diff_raw.to_frame(), df_ma_diff], axis=1)

# CRITICAL: Convert any remaining pd.NA to np.nan for matplotlib compatibility
df_ax_ma = df_ax_ma.fillna(np.nan)
df_flowdiff = df_flowdiff.fillna(np.nan)

# Align left and right frames to a common union index (sorted)
union_idx = df_ax_ma.index.union(df_flowdiff.index)
try:
    union_idx = union_idx.unique()
except Exception:
    pass
union_idx = union_idx.sort_values()

df_ax_ma = df_ax_ma.reindex(union_idx)
df_flowdiff = df_flowdiff.reindex(union_idx)

# Optional: sanity prints
print("Aligned lengths (L, R):", len(df_ax_ma.index), len(df_flowdiff.index))
print("Left NaNs total:", int(df_ax_ma.isna().sum().sum()))
print("Right NaNs total:", int(df_flowdiff.isna().sum().sum()))
print("Diff columns:", df_flowdiff.columns.tolist())
print("Non-NaN counts per diff column:\n", df_flowdiff.count())

In [ ]:
# Reorder df_flows columns so MA columns follow their corresponding raw data columns
# Strategy: For each raw column in df_ax, add it and then add all its MA variants

# Get list of original df_ax columns (before MA calculation)
raw_columns = [col for col in df_ax_ma.columns if '_MA_' not in col]

# Build ordered column list: for each raw column, add it plus all its MA variants
ordered_columns = []
for raw_col in raw_columns:
    # Add the raw column
    ordered_columns.append(raw_col)
    # Add all MA columns for this raw column
    for w_label in windows.keys():
        ma_col = f"{raw_col}_MA_{w_label}"
        if ma_col in df_ax_ma.columns:
            ordered_columns.append(ma_col)

# Reorder df_flows
df_ax_ma = df_ax_ma[ordered_columns]

print(f"Reordered df_flows columns ({len(df_ax_ma.columns)} total):")
print("First 20 columns:", df_ax_ma.columns[:20].tolist())
print("...")
print("Last 10 columns:", df_ax_ma.columns[-10:].tolist())

# Button Text Length Analysis
Analyze the total length of checkbox button texts and their relationship to the number of rows used in the InteractivePlotWindow frames.

In [ ]:
# # Import and run the button text analysis
# from calculate_button_text_metrics import analyze_button_layout

# # Analyze the dataframes that will be used in the interactive chart
# # df_flows = left axis (blue frame)
# # df_flowdiff = right axis (orange frame)
# results = analyze_button_layout(df_ax_ma, df_flowdiff)

# print("\n")
# print("=" * 80)
# print("SUMMARY FOR InteractivePlotWindow")
# print("=" * 80)
# print(f"✓ InteractivePlotWindow now uses OPTIMIZED LAYOUT!")
# print()
# print(f"Row optimization:")
# print(f"  Left frame will use: {results['optimal_rows_left']} rows")
# print(f"  Right frame will use: {results['optimal_rows_right']} rows")
# print()
# print(f"Width optimization:")
# print(f"  Left frame: {results['left']['num_cols']} columns → {results['left']['num_cols']}x stretch")
# print(f"  Right frame: {results['right']['num_cols']} columns → {results['right']['num_cols']}x stretch")
# print(f"  Horizontal space allocated proportionally to minimize blank space!")
# print()
# print("The layout algorithm:")
# print("  - Analyzes total character length of button texts")
# print("  - Calculates optimal rows (targets ~7 buttons per row)")
# print("  - Constrains to min 2 rows, max 4 rows")
# print("  - Fills VERTICALLY: row 0, row 1, ..., then next column")
# print("  - Allocates horizontal width based on column count ratio")
# print("=" * 80)

# Add elevation levels to df_ax_ma

In [ ]:
df_ax_ma = pd.merge(df_ax_ma,df_PlusNivåer, left_index=True, right_index=True, how='outer')

# Interactive Flow balance chart

In [ ]:
df_flows_ma = df_ax_ma.filter(like='[m3/h]')

In [ ]:
from InteractivePlotWindow import InteractivePlotWindow as IPW
from PyQt6.QtWidgets import QApplication
runchart = 0
if runchart == 1:
    if __name__ == "__main__":
        import sys as _sys
        from IPython import get_ipython

        def _make_and_show():
            app = QApplication.instance() or QApplication(_sys.argv)
            
            # Calculate total number of series for settings file key
            n_series_flows = len(df_flows_ma.columns) + len(df_flowdiff.columns)
            settings_key = f'Flöden och Flödesdiff ({n_series_flows} series)'
            print(f"Chart settings key: {settings_key}")
            
            mainWin = IPW(df_axL = df_flows_ma,
                                df_axL_Title = 'Flöde & Bräddflöde [m3/h]', 
                                df_axR = df_flowdiff, 
                                df_axR_Title = 'Flödesdiff [%]',
                                WindowTitle='Pajala ARV Flöde',
                                settings_file=f'InteractivePlotWindow.json::{settings_key}'
                            )
            mainWin.show()
            # Keep references to avoid garbage collection in notebook kernels.
            
            # Store on the app and module globals so the objects persist after this function returns.
            try:
                app._pajala_mainWin = mainWin
            except Exception:
                pass
            globals()['_pajala_mainWin'] = mainWin
            globals()['_pajala_app'] = app
            return app

        # If running inside an IPython kernel (notebook), request IPython to enable the Qt event loop
        if 'ipykernel' in _sys.modules:
            try:
                ip = get_ipython()
                if ip is not None:
                    # enable GUI event loop integration; this avoids a blocking app.exec() call
                    ip.run_line_magic('gui', 'qt')
            except Exception:
                ip = None
            # Create and show window but do NOT call app.exec() - the event loop is managed by IPython
            app = _make_and_show()
            # Keep references in the IPython user namespace if available so users can interact with them
            if ip is not None:
                try:
                    ip.user_ns['_pajala_app'] = app
                    ip.user_ns['_pajala_mainWin'] = globals().get('_pajala_mainWin')
                except Exception:
                    # Fall back to module globals (already set by _make_and_show)
                    pass
        else:
            # Running as a script: start the blocking event loop
            app = _make_and_show()
            _sys.exit(app.exec())

# Sludge Treatment Chart

In [ ]:
import math
# Delete variables if they exist (more robust method)
# Delete variables if they exist (compact version)
for var_name in ['df_sludge_ma', 'df_sludgeflow_ma', 'df_sludgelevel_ma']:
    try:
        exec(f'del {var_name}')
    except NameError:
        pass
df_sludge_ma = df_ax_ma.filter(regex='Slam|Returslam|AvvatPoly|Slam|Puck|Exten|Rejekt|FlytSlam|[m]').copy()
df_sludge_ma['Inflöde Extenslam FT80101 [m3/h]'] = df_AllaFlöden[['Inflöde Extenslam FT80101 [m3/h]']].copy()
df_sludge_ma['Inflöde Extenslam FT80101 uncalib [m3/h]'] = df_AllaFlöden[['Inflöde Extenslam FT80101 uncalib [m3/h]']].copy()
df_sludge_ma['Inflöde Extenslam FT80101 [m3]'] = (df_sludge_ma['Inflöde Extenslam FT80101 [m3/h]']/60).cumsum()
df_sludge_ma['Inflöde Extenslam FT80101 uncalib [m3]'] = (df_sludge_ma['Inflöde Extenslam FT80101 uncalib [m3/h]']/60).cumsum()
df_sludge_ma['FlytSlam LT21101 [m*m2]'] = df_sludge_ma['FlytSlam LT21101 [m]'] * 1.25*2
df_sludge_ma['SFÖverSlam LT71101 [m]'] = df_Skivfilter['SFÖverSlam LT71101 [m]']
df_sludge_ma['SFÖverSlam LT71101 [m*m2]'] = df_Skivfilter['SFÖverSlam LT71101 [m]'] * 1.5*2.2
df_sludge_ma['Slamlager 1 LT81101 [m*m2]'] = df_sludge_ma['Slamlager 1 LT81101 [m]'] * 5*7.5
df_sludge_ma['Slamlager 2 LT82101 [m*m2]'] = df_sludge_ma['Slamlager 2 LT82101 [m]'] * 3*3.5
df_sludge_ma['FörsedUtlopp LT20103 [m]'] = df_Försedimentering['FörsedUtlopp LT20103 [m]']
df_sludge_ma['Rejekt LT84101 [m*m2]'] = df_sludge_ma['Rejekt LT84101 [m]'] * 1.2**2/4*math.pi
df_sludge_ma['SFIn LT70201 [m]'] = df_Skivfilter['SFIn LT70201 [m]']
df_sludge_ma['SFUt LT70102 [m]'] = df_Skivfilter['SFUt LT70102 [m]']
df_sludge_ma['SFIn LT70201 [m]'] = df_Skivfilter['SFIn LT70201 [m]']
df_sludge_ma['SFUt LT70202 [m]'] = df_Skivfilter['SFUt LT70202 [m]']
df_sludge_ma = df_sludge_ma.drop(columns=df_sludge_ma.filter(regex=r'\[m3/h\]_MA').columns.tolist())
df_sludge_ma = df_sludge_ma.drop(columns=df_sludge_ma.filter(regex=r'uncalibrated').columns.tolist())
df_sludge_ma = df_sludge_ma.drop(columns=df_sludge_ma.filter(regex=r'\[l/h\]_MA').columns.tolist())
df_sludge_ma = df_sludge_ma.drop(columns=df_sludge_ma.filter(regex=r'\[1/0\]').columns.tolist())
df_sludge_ma = df_sludge_ma.drop(columns=df_sludge_ma.filter(regex=r'\[Hz\]').columns.tolist())
df_sludge_ma = df_sludge_ma.drop(columns=df_sludge_ma.filter(regex=r'\[m\]_MA').columns.tolist())
df_sludge_ma = df_sludge_ma.drop(columns=df_sludge_ma.filter(regex=r'\[ppm]').columns.tolist())
# df_sludgeflow_ma = df_sludge_ma.filter(regex=r'\[m3/h\]')
# df_sludgelevel_ma = df_sludge_ma.filter(regex=r'\[m\]')


In [ ]:
# #Rejektvatten tank nivå analys -  Försök till att separera inflöde och utflöde baserat på nivåändringar (men detta stämmer inte helt när cyklerna är närma uppdateringsintervallet 1 minut)
# import pandas as pd
# for var_name in ['df_rejekt']:
#     try:
#         exec(f'del {var_name}')
#     except NameError:
#         pass
# # Calculate the difference between consecutive level readings
# df_rejekt = df_sludge_ma[['Rejekt LT84101 [m*m2]']].copy()
# df_rejekt['Volym_Delta'] = df_rejekt['Rejekt LT84101 [m*m2]'].diff()

# # Classify periods
# df_rejekt['Trend'] = pd.NA
# df_rejekt.loc[df_rejekt['Volym_Delta'] < 0, 'Trend'] = 'Decreasing'   # Likely pumping
# df_rejekt.loc[df_rejekt['Volym_Delta'] > 0, 'Trend'] = 'Increasing'   # Likely filling
# df_rejekt.loc[df_rejekt['Volym_Delta'] == 0, 'Trend'] = 'Stable'

# df_rejekt.loc[df_rejekt['Volym_Delta'] > 0, 'Inflöde Rejekttank [m*m2]'] = df_rejekt['Volym_Delta']
# df_rejekt.loc[df_rejekt['Volym_Delta'] < 0, 'Utflöde Rejekttank [m*m2]'] = -df_rejekt['Volym_Delta']

# df_rejekt.loc[df_rejekt['Volym_Delta'] > 0, 'Rejekttank [m3]'] = df_rejekt['Volym_Delta']
# df_rejekt.loc[df_rejekt['Volym_Delta'] < 0, 'Rejekttank [m3]'] = -df_rejekt['Volym_Delta']
# df_rejekt.loc[df_rejekt['Volym_Delta'] == 0, 'Rejekttank [m3]'] = 0

# # Optional: Show only periods with change
# df_changes = df_rejekt[df_rejekt['Trend'] != 'Stable']

# df_rejekt['Inflöde Rejekttank [m3]'] = df_rejekt['Inflöde Rejekttank [m*m2]'].cumsum()
# df_rejekt['Utflöde Rejekttank [m3]'] = df_rejekt['Utflöde Rejekttank [m*m2]'].cumsum()
# df_rejekt['Inflöde Rejekttank [m3/h]'] = df_rejekt['Inflöde Rejekttank [m*m2]'] * 60
# # df_rejekt.loc[df_rejekt['Inflöde Rejekttank [m*m2]'] != np.nan, 'Inflöde Rejekttank [m3/h]'] = df_rejekt['Volym_Delta'] * 60
# # df_rejekt.loc[df_rejekt['Utflöde Rejekttank [m*m2]'] != np.nan, 'Utflöde Rejekttank [m3/h]'] = df_rejekt['Volym_Delta'] * 60

# df_sludge_ma = pd.merge(df_sludge_ma, df_rejekt[['Inflöde Rejekttank [m3/h]','Inflöde Rejekttank [m3]', 'Utflöde Rejekttank [m3]']], left_index=True, right_index=True, how='left')

# print(df_changes.head(20))
# print(df_rejekt.head(20))

In [ ]:
# Calculate the difference between consecutive level readings
for var_name in ['df_slamlager_2']:
    try:
        exec(f'del {var_name}')
    except NameError:
        pass
df_slamlager_2 = df_sludge_ma[['Slamlager 2 LT82101 [m*m2]']].copy()
df_slamlager_2['Volym_Delta [m3/min]'] = df_slamlager_2['Slamlager 2 LT82101 [m*m2]'].diff()
df_slamlager_2['Inflöde Slamlager 2 [m3/h]'] = df_slamlager_2['Volym_Delta [m3/min]']*60 - df_sludge_ma['SlamAvvattn FT82101 [m3/h]']
df_slamlager_2['Inflöde Rejekttank Puck [m3/h]'] = df_sludge_ma['PuckS FT81101 [m3/h]'].copy() - df_slamlager_2['Inflöde Slamlager 2 [m3/h]'] 

# Calculate 10-minute moving average (centered on each point)
df_slamlager_2['Inflöde Slamlager 2 [m3/h]_MA_10min'] = (
    df_slamlager_2['Inflöde Slamlager 2 [m3/h]']
    .rolling('10min', min_periods=1, center=True).mean()
)

# Calculate 10-minute moving average (centered on each point)
df_slamlager_2['Inflöde Rejekttank Puck [m3/h]_MA_10min'] = (
    df_slamlager_2['Inflöde Rejekttank Puck [m3/h]']
    .rolling('10min', min_periods=1, center=True).mean()
)
df_sludge_ma = pd.merge(df_sludge_ma, df_slamlager_2[['Inflöde Slamlager 2 [m3/h]','Inflöde Slamlager 2 [m3/h]_MA_10min','Inflöde Rejekttank Puck [m3/h]','Inflöde Rejekttank Puck [m3/h]_MA_10min']], left_index=True, right_index=True, how='left')
# df_slamlager_2.loc[df_slamlager_2['Volym_Delta [m3/min]'] > 0, 'Inflöde Rejekttank [m3/h]'] = df_slamlager_2['Volym_Delta [m3/min]']
# df_slamlager_2.loc[df_slamlager_2['Volym_Delta [m3/min]'] < 0, 'Utflöde Rejekttank [m3/h]'] = -df_slamlager_2['Volym_Delta [m3/min]']
# df_slamlager_2.loc[df_slamlager_2['Volym_Delta [m3/min]'] == 0, 'Utflöde Rejekttank [m3/h]'] = 0

# Optional: Show only periods with change
# df_changes = df_slamlager_2[df_slamlager_2['Trend'] != 'Stable']

# df_slamlager_2['Inflöde Rejekttank [m3]'] = df_slamlager_2['Inflöde Rejekttank [m*m2]'].cumsum()
# df_slamlager_2['Utflöde Rejekttank [m3]'] = df_slamlager_2['Utflöde Rejekttank [m*m2]'].cumsum()
# df_slamlager_2['Inflöde Rejekttank [m3/h]'] = df_slamlager_2['Inflöde Rejekttank [m*m2]'] * 60

In [ ]:
# Diagram över Slamflöden och Slamnivåer
from InteractivePlotWindowMultiAxis import InteractivePlotWindowMultiAxis as IPWMultiAxis
from PyQt6.QtWidgets import QApplication

runchart = 1
if runchart == 1:
    if __name__ == "__main__":
        import sys as _sys
        from IPython import get_ipython

        def _make_and_show():
            app = QApplication.instance()
            if app is None:
                app = QApplication(_sys.argv)
            
            # Calculate total number of series for settings file key
            n_series_flows = len(df_sludge_ma.columns)
            settings_key = f'Pajala ARV Slamflöden och Slamnivåer ({n_series_flows} series)'
            print(f"Chart settings key: {settings_key}")

            mainWin = IPWMultiAxis(df_sludge_ma, WindowTitle="Pajala ARV Slamflöden och Slamnivåer")
            mainWin.show()
            
            # Keep references to avoid garbage collection
            try:
                app._pajala_mainWin = mainWin
            except Exception:
                pass
            globals()['_pajala_mainWin'] = mainWin
            globals()['_pajala_app'] = app
            return app

        # If running inside an IPython kernel (notebook)
        if 'ipykernel' in _sys.modules:
            try:
                ip = get_ipython()
                if ip is not None:
                    # Enable GUI event loop integration
                    ip.run_line_magic('gui', 'qt')
            except Exception:
                ip = None
            
            # Create and show window (no app.exec())
            app = _make_and_show()
            
            # Keep references in IPython user namespace
            if ip is not None:
                try:
                    ip.user_ns['_pajala_app'] = app
                    ip.user_ns['_pajala_mainWin'] = globals().get('_pajala_mainWin')
                except Exception:
                    pass
        else:
            # Running as a script: blocking event loop
            app = _make_and_show()
            _sys.exit(app.exec())

# Disc Filter Chart

In [ ]:
import math
# Delete variables if they exist (compact version)
for var_name in ['df_discfilter_ma']:
    try:
        exec(f'del {var_name}')
    except NameError:
        pass
df_discfilter_ma = df_ax_ma.filter(regex=r'flöde|\[\+m\]').copy()
# df_discfilter_ma['Inflöde Extenslam FT80101 [m3/h]'] = df_AllaFlöden[['Inflöde Extenslam FT80101 [m3/h]']]
# df_discfilter_ma['Inflöde Extenslam FT80101 [m3]'] = (df_discfilter_ma['Inflöde Extenslam FT80101 [m3/h]']/60).cumsum()
# df_discfilter_ma['SFÖverSlam LT71101 [m]'] = df_Skivfilter['SFÖverSlam LT71101 [m]']
df_discfilter_ma['SFÖverSlam LT71101 [m*m2]'] = df_Skivfilter['SFÖverSlam LT71101 [m]'] * 1.5*2.2
# df_discfilter_ma['FörsedUtlopp LT20103 [m]'] = df_Försedimentering['FörsedUtlopp LT20103 [m]']
# df_discfilter_ma['Rejekt LT84101 [m*m2]'] = df_discfilter_ma['Rejekt LT84101 [m]'] * 1.2**2/4*math.pi

# df_discfilter_ma['SFIN LT70101 [m]'] = df_Skivfilter['SFIn LT70101 [m]']
# df_discfilter_ma['SFUt LT70102 [m]'] = df_Skivfilter['SFUt LT70102 [m]']
# df_discfilter_ma['SFIn LT70201 [m]'] = df_Skivfilter['SFIn LT70201 [m]']
# df_discfilter_ma['SFUt LT70202 [m]'] = df_Skivfilter['SFUt LT70202 [m]']
# df_discfilter_ma['MBBR LT31101 [m]'] = df_MBBR['MBBR LT31101 [m]']
# df_discfilter_ma['MBBR LT31201 [m]'] = df_MBBR['MBBR LT31201 [m]']

# df_discfilter_ma = df_discfilter_ma.drop(columns=df_discfilter_ma.filter(regex=r'\[m3/h\]_MA').columns.tolist())
df_discfilter_ma = df_discfilter_ma.drop(columns=df_discfilter_ma.filter(regex=r'uncalibrated').columns.tolist())
# df_discfilter_ma = df_discfilter_ma.drop(columns=df_discfilter_ma.filter(regex=r'\[l/h\]_MA').columns.tolist())
df_discfilter_ma = df_discfilter_ma.drop(columns=df_discfilter_ma.filter(regex=r'\[1/0\]').columns.tolist())
df_discfilter_ma = df_discfilter_ma.drop(columns=df_discfilter_ma.filter(regex=r'\[Hz\]').columns.tolist())
df_discfilter_ma = df_discfilter_ma.drop(columns=df_discfilter_ma.filter(regex=r'\[m\]_MA').columns.tolist())
df_discfilter_ma = df_discfilter_ma.drop(columns=df_discfilter_ma.filter(regex=r'\[m\]').columns.tolist())
df_discfilter_ma = df_discfilter_ma.drop(columns=df_discfilter_ma.filter(regex=r'\[\+m\]_MA').columns.tolist())
# df_sludgeflow_ma = df_sludge_ma.filter(regex=r'\[m3/h\]')
# df_sludgelevel_ma = df_sludge_ma.filter(regex=r'\[m\]')


In [ ]:
# Diagram över Skivfilter flöden och nivåer
from InteractivePlotWindowMultiAxis import InteractivePlotWindowMultiAxis as IPWMultiAxis
from PyQt6.QtWidgets import QApplication

runchart = 0
if runchart == 1:
    if __name__ == "__main__":
        import sys as _sys
        from IPython import get_ipython

        def _make_and_show():
            app = QApplication.instance()
            if app is None:
                app = QApplication(_sys.argv)
            
            # Calculate total number of series for settings file key
            n_series_flows = len(df_discfilter_ma.columns)
            settings_key = f'Pajala ARV Skivfilter ({n_series_flows} series)'
            print(f"Chart settings key: {settings_key}")

            mainWin = IPWMultiAxis(df_discfilter_ma, WindowTitle="Pajala ARV Skivfilter")
            mainWin.show()
            
            # Keep references to avoid garbage collection
            try:
                app._pajala_mainWin = mainWin
            except Exception:
                pass
            globals()['_pajala_mainWin'] = mainWin
            globals()['_pajala_app'] = app
            return app

        # If running inside an IPython kernel (notebook)
        if 'ipykernel' in _sys.modules:
            try:
                ip = get_ipython()
                if ip is not None:
                    # Enable GUI event loop integration
                    ip.run_line_magic('gui', 'qt')
            except Exception:
                ip = None
            
            # Create and show window (no app.exec())
            app = _make_and_show()
            
            # Keep references in IPython user namespace
            if ip is not None:
                try:
                    ip.user_ns['_pajala_app'] = app
                    ip.user_ns['_pajala_mainWin'] = globals().get('_pajala_mainWin')
                except Exception:
                    pass
        else:
            # Running as a script: blocking event loop
            app = _make_and_show()
            _sys.exit(app.exec())

# MBBR Chart

In [ ]:
# Delete variables if they exist (compact version)
for var_name in ['df_MBBR_ma', 'df_MBBRflow_ma', 'df_MBBRlevel_ma']:
    try:
        exec(f'del {var_name}')
    except NameError:
        pass    
df_MBBR_ma = df_ax_ma.filter(regex='flöde|MBBR', axis=1)
# df_MBBRflow_ma = df_MBBR_ma.filter(regex=r'\[m3/h\]')
# df_MBBRlevel_ma = df_MBBR_ma.filter(regex=r'\[m\]')

In [ ]:
from InteractivePlotWindowMultiAxis import InteractivePlotWindowMultiAxis as IPWMultiAxis
from PyQt6.QtWidgets import QApplication
import sys

runchart = 0
if runchart == 1:
    if __name__ == "__main__":
        import sys as _sys
        from IPython import get_ipython

        def _make_and_show():
            app = QApplication.instance() or QApplication(_sys.argv)
            
            # Calculate total number of series for settings file key
            n_series_flows = len(df_MBBR_ma.columns) # + len(df_sludgelevel_ma.columns)
            settings_key = f'Pajala ARV MBBR ({n_series_flows} series)'
            print(f"Chart settings key: {settings_key}")

            app = QApplication(sys.argv)
            mainWin = IPWMultiAxis(df_MBBR_ma, WindowTitle="Multi-Unit Monitoring Test")
            mainWin.show()
            mainWin.show()
            # Keep references to avoid garbage collection in notebook kernels.
            
            # Store on the app and module globals so the objects persist after this function returns.
            try:
                app._pajala_mainWin = mainWin
            except Exception:
                pass
            globals()['_pajala_mainWin'] = mainWin
            globals()['_pajala_app'] = app
            return app

        # If running inside an IPython kernel (notebook), request IPython to enable the Qt event loop
        if 'ipykernel' in _sys.modules:
            try:
                ip = get_ipython()
                if ip is not None:
                    # enable GUI event loop integration; this avoids a blocking app.exec() call
                    ip.run_line_magic('gui', 'qt')
            except Exception:
                ip = None
            # Create and show window but do NOT call app.exec() - the event loop is managed by IPython
            app = _make_and_show()
            # Keep references in the IPython user namespace if available so users can interact with them
            if ip is not None:
                try:
                    ip.user_ns['_pajala_app'] = app
                    ip.user_ns['_pajala_mainWin'] = globals().get('_pajala_mainWin')
                except Exception:
                    # Fall back to module globals (already set by _make_and_show)
                    pass
        else:
            # Running as a script: start the blocking event loop
            app = _make_and_show()
            _sys.exit(app.exec())

# Interactive Velocity Chart (FT10101)
Interactive plot showing flow velocity in m/s with flow differences on the right axis.

In [ ]:
# from InteractivePlotWindow import InteractivePlotWindow as IPW
# from PyQt6.QtWidgets import QApplication

# Align the velocity (with MAs) and flowdiff DataFrames to the same index
union_idx_velocity = df_velocity_with_ma.index.union(df_flowdiff.index)
df_velocity_aligned = df_velocity_with_ma.reindex(union_idx_velocity)
df_flowdiff_aligned = df_flowdiff.reindex(union_idx_velocity)

print(f"Aligned velocity DataFrame shape: {df_velocity_aligned.shape}")
print(f"Aligned flowdiff DataFrame shape: {df_flowdiff_aligned.shape}")

runchart = 0
if runchart == 1:
    if __name__ == "__main__":
        import sys as _sys
        from IPython import get_ipython

        def _make_and_show_velocity():
            app = QApplication.instance() or QApplication(_sys.argv)
            
            # Calculate total number of series for settings file key
            n_series_velocity = len(df_velocity_aligned.columns) + len(df_flowdiff_aligned.columns)
            settings_key_velocity = f'Hastighet FT10101 och Flödesdiff ({n_series_velocity} series)'
            print(f"Chart settings key: {settings_key_velocity}")
            
            mainWin_velocity = IPW(
                df_axL = df_velocity_aligned,
                df_axL_Title = 'Hastighet FT10101 [m/s]', 
                df_axR = df_flowdiff_aligned, 
                df_axR_Title = 'Flödesdiff [%]',
                WindowTitle='Pajala ARV - Hastighet FT10101 och Flödesdiff',
                settings_file=f'InteractivePlotWindow.json::{settings_key_velocity}'
            )
            mainWin_velocity.show()
            # Keep references to avoid garbage collection in notebook kernels.
            
            # Store on the app and module globals so the objects persist after this function returns.
            try:
                app._pajala_velocity_mainWin = mainWin_velocity
            except Exception:
                pass
            globals()['_pajala_velocity_mainWin'] = mainWin_velocity
            globals()['_pajala_velocity_app'] = app
            return app

        # If running inside an IPython kernel (notebook), request IPython to enable the Qt event loop
        if 'ipykernel' in _sys.modules:
            try:
                ip = get_ipython()
                if ip is not None:
                    # enable GUI event loop integration; this avoids a blocking app.exec() call
                    ip.run_line_magic('gui', 'qt')
            except Exception:
                ip = None
            # Create and show window but do NOT call app.exec() - the event loop is managed by IPython
            app = _make_and_show_velocity()
            # Keep references in the IPython user namespace if available so users can interact with them
            if ip is not None:
                try:
                    ip.user_ns['_pajala_velocity_app'] = app
                    ip.user_ns['_pajala_velocity_mainWin'] = globals().get('_pajala_velocity_mainWin')
                except Exception:
                    # Fall back to module globals (already set by _make_and_show_velocity)
                    pass
        else:
            # Running as a script: start the blocking event loop
            app = _make_and_show_velocity()
            _sys.exit(app.exec())

# Flow Difference vs Velocity Correlation Chart
Cross-plot showing the relationship between flow velocity and flow differences.

In [ ]:
# Create 4 separate XY scatter plots: Velocity (X) vs Flow Difference (Y)
# 1. Raw data (no MA)
# 2. MA_1h vs MA_1h
# 3. MA_24h vs MA_24h
# 4. MA_7d vs MA_7d

import matplotlib.pyplot as plt

# Define the pairs to plot: (velocity_column, flowdiff_column, title_suffix)
plot_pairs = [
    ('Inflöde FT10101 [m/s]', 'Diff_MA_1h', 'Raw Data'),
    ('Inflöde FT10101 [m/s]_MA_1h', 'Diff_MA_1h', 'MA 1h'),
    ('Inflöde FT10101 [m/s]_MA_24h', 'Diff_MA_24h', 'MA 24h'),
    ('Inflöde FT10101 [m/s]_MA_7d', 'Diff_MA_7d', 'MA 7d')
]

# Create 2x2 subplot layout
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, (velocity_col, flowdiff_col, title_suffix) in enumerate(plot_pairs):
    ax = axes[idx]
    dprint(flowdiff_col)
    # Get non-NaN pairs only
    mask = df_velocity_aligned[velocity_col].notna() & df_flowdiff_aligned[flowdiff_col].notna()
    x_data = df_velocity_aligned.loc[mask, velocity_col]
    y_data = df_flowdiff_aligned.loc[mask, flowdiff_col]
    
    # Create scatter plot
    ax.scatter(x_data, y_data, alpha=0.3, s=1, color='blue')
    
    ax.set_xlabel('Hastighet FT10101 [m/s]', fontsize=11)
    ax.set_ylabel('Flödesdiff [%]', fontsize=11)
    ax.set_title(f'Flödesdiff vs Hastighet - {title_suffix}', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    # Add data point count
    ax.text(0.02, 0.98, f'n = {mask.sum():,}', 
            transform=ax.transAxes, 
            verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print("Created 4 scatter plots:")
print("  1. Raw velocity vs Diff_MA_1h")
print("  2. Velocity MA_1h vs Diff_MA_1h")
print("  3. Velocity MA_24h vs Diff_MA_24h")
print("  4. Velocity MA_7d vs Diff_MA_7d")

In [ ]:
# # Cross-Correlation Analysis
# from scipy.signal import correlate
# import numpy as np

# # Calculate cross-correlation
# correlation = correlate(df_ax['Inflöde FT10101'].fillna(0), 
#                         df_ax['Utflöde FT72101'].fillna(0), 
#                         mode='full')

# # Find the lag that gives maximum correlation
# lags = np.arange(-len(df_ax) + 1, len(df_ax))
# optimal_lag_idx = np.argmax(correlation)
# optimal_lag = lags[optimal_lag_idx]

# print(f"Optimal lag: {optimal_lag} minutes")

# Cumulative Flow Charts

In [ ]:
# Calculate cumulative volumes
df_cumsumflows = pd.DataFrame(index=df_ax.index)
df_cumsumflows['Inflöde FT10101_[m3]]'] = df_ax['Inflöde FT10101 [m3/h]'].cumsum()
df_cumsumflows['Inflöde Extenslam FT80101_[m3]'] = df_ax['Inflöde Extenslam FT80101 [m3/h]'].cumsum()
df_cumsumflows['Utflöde FT72101_[m3]'] = df_ax['Utflöde FT72101 [m3/h]'].cumsum()
df_cumsumflows['MBBRflöde FT30101_[m3]'] = df_ax['MBBRflöde FT30101 [m3/h]'].cumsum()
df_cumsumflows['Utflöde Bräddning LT23101_[m3]'] = df_ax['Utflöde Bräddning LT23101 [m3/h]'].cumsum()

df_cumsumflows['Inflow_[m3]'] = (df_ax['Inflöde FT10101 [m3/h]'] + 
                          df_ax['Inflöde Extenslam FT80101 [m3/h]']).cumsum()
df_cumsumflows['Outflow_[m3]'] = (df_ax['Utflöde FT72101 [m3/h]'] + 
                           df_ax['Utflöde Bräddning LT23101 [m3/h]']).cumsum()

# Storage change = Cumulative inflow - Cumulative outflow
df_cumsumflows['Storage_Change_[m3]'] = df_cumsumflows['Inflow_[m3]'] - df_cumsumflows['Outflow_[m3]']
df_cumsumflows['Storage_Change_Percent'] = df_cumsumflows['Storage_Change_[m3]'] / df_cumsumflows['Inflow_[m3]'] * 100.0
# df_flow_cumsum = pd.merge(df_flows,df_cumsumflows, left_index=True, right_index=True, how='outer'                          )

In [ ]:
from InteractivePlotWindow import InteractivePlotWindow as IPW
from PyQt6.QtWidgets import QApplication

if False:
    if __name__ == "__main__":
        import sys as _sys
        from IPython import get_ipython

        def _make_and_show():
            app = QApplication.instance() or QApplication(_sys.argv)
            
            # Calculate total number of series for settings file key
            n_series_flows = len(df_ax_ma.columns) + len(df_flowdiff.columns)
            settings_key = f'Kumsumflöden in och ut ({n_series_flows} series)'
            print(f"Chart settings key: {settings_key}")
            
            mainWin = IPW(df_axL = df_ax_ma,
                                df_axL_Title = 'Flöde [m3/h]', 
                                df_axR = df_cumsumflows, 
                                df_axR_Title = 'Kumsumflöde [m3]',
                                WindowTitle='Pajala ARV Flöden och kumsumflöden',
                                settings_file=f'InteractivePlotWindow.json::{settings_key}'
                            )
            mainWin.show()
            # Keep references to avoid garbage collection in notebook kernels.
            
            # Store on the app and module globals so the objects persist after this function returns.
            try:
                app._pajala_mainWin = mainWin
            except Exception:
                pass
            globals()['_pajala_mainWin'] = mainWin
            globals()['_pajala_app'] = app
            return app

        # If running inside an IPython kernel (notebook), request IPython to enable the Qt event loop
        if 'ipykernel' in _sys.modules:
            try:
                ip = get_ipython()
                if ip is not None:
                    # enable GUI event loop integration; this avoids a blocking app.exec() call
                    ip.run_line_magic('gui', 'qt')
            except Exception:
                ip = None
            # Create and show window but do NOT call app.exec() - the event loop is managed by IPython
            app = _make_and_show()
            # Keep references in the IPython user namespace if available so users can interact with them
            if ip is not None:
                try:
                    ip.user_ns['_pajala_app'] = app
                    ip.user_ns['_pajala_mainWin'] = globals().get('_pajala_mainWin')
                except Exception:
                    # Fall back to module globals (already set by _make_and_show)
                    pass
        else:
            # Running as a script: start the blocking event loop
            app = _make_and_show()
            _sys.exit(app.exec())